In [1]:
# Importing the Dependancies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [3]:
# First 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [4]:
# Last 5 rows of the dataset
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
13949,24754,1.252924,-0.182189,-0.802716,-0.210981,1.916713,3.643624,-0.778711,0.818295,1.706962,-0.539481,0.524218,-2.224983,1.506368,1.469584,-0.670082,0.122806,0.247449,-0.019811,0.418664,-0.001336,-0.497088,-1.211285,0.043809,0.964159,0.442030,0.261483,-0.051402,0.005112,23.74,0.0
13950,24756,-0.346979,-2.103284,-0.685061,1.961605,-0.401125,0.473632,1.133816,-0.256528,0.893409,-0.639733,2.069409,-1.105807,2.339853,1.921904,-1.153485,-0.336047,0.509636,0.060266,-0.535280,1.437054,0.359662,-0.316275,-0.864259,-0.279881,0.491802,-0.353996,-0.149931,0.129795,794.20,0.0
13951,24759,-6.053652,-5.988723,0.810413,-0.011811,1.308135,-0.590803,-0.725838,-0.234840,1.624646,0.816619,2.557427,-2.480679,1.757415,0.307835,-0.667360,1.424542,0.717358,-1.323944,-0.091943,-2.982379,-0.771970,1.474668,3.176363,-0.302410,0.052529,-0.373871,-0.700463,2.508443,60.00,0.0
13952,24759,1.169121,-1.284945,0.032717,-0.681670,0.660598,4.412578,-1.913115,1.076592,1.501230,0.089684,-0.085081,-2.392619,1.877502,0.643433,-1.206728,-1.262256,0.563295,1.845204,-0.471718,-0.265041,-0.557596,-0.882435,-0.041523,0.975445,0.297229,0.550515,0.015029,0.032067,90.00,0.0
13953,24759,-6.917152,5.854171,-1.652458,-1.488884,-0.833891,-0.344418,0.393789,0.379968,6.133597,8.063261,3.074405,-1.822549,1.637008,-1.010715,0.231682,-0.349489,0.262064,-0.188737,-1.008723,3.469163,-1.404681,-1.124694,0.174333,-0.528234,0.990685,-0.035875,1.071374,-0.168831,NaN,NaN


In [5]:
# Dataset information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13954 entries, 0 to 13953
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    13954 non-null  int64  
 1   V1      13954 non-null  float64
 2   V2      13954 non-null  float64
 3   V3      13954 non-null  float64
 4   V4      13954 non-null  float64
 5   V5      13954 non-null  float64
 6   V6      13954 non-null  float64
 7   V7      13954 non-null  float64
 8   V8      13954 non-null  float64
 9   V9      13954 non-null  float64
 10  V10     13954 non-null  float64
 11  V11     13954 non-null  float64
 12  V12     13954 non-null  float64
 13  V13     13954 non-null  float64
 14  V14     13954 non-null  float64
 15  V15     13954 non-null  float64
 16  V16     13954 non-null  float64
 17  V17     13954 non-null  float64
 18  V18     13954 non-null  float64
 19  V19     13954 non-null  float64
 20  V20     13954 non-null  float64
 21  V21     13954 non-null  float64
 22

In [6]:
# Checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    1
Class     1
dtype: int64

In [7]:
# Distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    13897
1.0       56
Name: Class, dtype: int64

Highly unbalanced Dataset. 

0 -> Normal Transaction

1 -> Fraudulent Transaction

In [8]:
# Separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [9]:
print(legit.shape)
print(fraud.shape)

(13897, 31)
(56, 31)


In [10]:
# Statistical measures of the data
legit.Amount.describe()

count    13897.000000
mean        63.773909
std        177.164503
min          0.000000
25%          5.770000
50%         15.900000
75%         52.370000
max       7712.430000
Name: Amount, dtype: float64

In [11]:
fraud.Amount.describe()

count      56.000000
mean       90.815893
std       310.308450
min         0.000000
25%         1.000000
50%         1.000000
75%         1.025000
max      1809.680000
Name: Amount, dtype: float64

In [12]:
# Compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,10111.481255,-0.217093,0.266252,0.887227,0.276789,-0.107510,0.136745,-0.131462,-0.020385,1.005460,-0.305324,0.841454,-1.524104,1.010368,0.886982,-0.190759,-0.019670,0.452113,0.011239,-0.087578,0.020758,-0.068686,-0.158230,-0.034834,0.013065,0.117928,0.034349,0.009565,0.002329,63.773909
1.0,12262.107143,-4.727948,4.660436,-9.328536,6.783464,-2.890388,-2.030360,-6.578337,1.028374,-2.981468,-7.502753,6.879584,-10.591544,0.548941,-9.973580,-0.256254,-5.225182,-7.478164,-2.516508,0.272975,0.453283,0.116896,0.048132,-0.225526,-0.452514,-0.124184,0.333325,0.793484,-0.046177,90.815893


**Under-Sampling**

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions.

(Number of Fraudulent Transactions --> 492)

In [13]:
legit_sample = legit.sample(n=492)

In [14]:
# Concatenating two DataFrames
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [15]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
6071,6899,-1.614750,1.513623,1.151692,1.248492,0.333332,0.250585,0.295978,0.149119,1.485890,0.965907,2.201480,-1.390262,1.824105,1.272731,-1.164372,-1.120464,1.043315,-0.055294,0.717698,0.394159,-0.333931,0.130308,-0.299131,-0.009373,0.048102,-0.252782,0.954628,0.398771,39.02,0.0
4077,3734,1.362114,-0.510953,0.557414,-0.633764,-0.948557,-0.509434,-0.814432,-0.040927,0.456547,0.438548,1.963358,-3.120383,0.568251,1.706052,-0.462122,1.558650,0.650460,-0.550711,0.822343,-0.003435,-0.221479,-0.639275,0.096812,-0.104656,0.173837,-0.514697,-0.031541,0.002629,22.82,0.0
7381,9950,1.127197,0.044301,0.833523,0.977531,-0.397751,0.124457,-0.380007,0.072450,1.584630,-0.467469,2.310519,-1.049775,1.703139,1.427401,-1.631381,-0.563792,0.909064,-0.344292,-0.164083,-0.176640,-0.142925,0.094908,-0.038306,0.231921,0.441474,0.398498,-0.027920,-0.006466,6.99,0.0
5559,5674,-0.547534,1.164591,3.170520,3.076056,-0.481031,0.783650,-0.194755,0.264877,0.025565,0.229945,2.036786,-2.010601,1.957894,1.286595,-0.551858,0.130144,0.632553,1.120590,0.795227,0.167287,0.093558,0.605005,-0.264680,0.508666,0.023524,0.321145,0.093608,0.085051,15.17,0.0
2353,1899,-0.994431,0.719533,1.662606,1.067349,-0.360177,1.549183,-0.090324,0.829148,-0.118648,-0.792222,-2.068326,-0.327677,0.099347,-0.081842,0.889558,-0.108800,0.099033,0.340293,1.070179,0.190340,0.028453,-0.000416,-0.121229,-1.341476,0.367044,-0.095339,-0.008284,0.021924,107.94,0.0


In [16]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
11880,20451,-15.819179,8.775997,-22.804686,11.864868,-9.092361,-2.386893,-16.560368,0.948349,-6.310658,-13.088891,9.815703,-14.056061,0.777192,-13.761018,-0.353636,-7.957447,-11.962954,-4.780508,0.652498,0.992279,-2.350634,1.036362,1.136051,-1.043414,-0.108923,0.657437,2.136424,-1.411945,1.0,1.0
12070,20931,-16.367923,9.223692,-23.270631,11.844777,-9.462037,-2.450444,-16.925152,1.384208,-6.287736,-13.002709,9.691461,-13.886595,0.838361,-13.517072,-0.377911,-7.855681,-11.803815,-4.761026,0.618624,0.993585,-2.343674,1.004602,1.188212,-1.047184,-0.035573,0.664900,2.122796,-1.416741,1.0,1.0
12108,21046,-16.917468,9.669900,-23.736443,11.824990,-9.830548,-2.514829,-17.290657,1.820408,-6.264903,-12.916636,9.567110,-13.717067,0.899541,-13.272965,-0.402260,-7.754094,-11.644603,-4.741303,0.584626,0.996745,-2.336111,0.972755,1.241866,-1.051086,0.038009,0.672317,2.108471,-1.421243,1.0,1.0
12261,21419,-17.467710,10.114816,-24.202142,11.805469,-10.198046,-2.579938,-17.656788,2.256902,-6.242149,-12.830657,9.442665,-13.547486,0.960729,-13.028717,-0.426674,-7.652662,-11.485328,-4.721370,0.550519,1.001519,-2.328024,0.940830,1.296817,-1.055104,0.111792,0.679695,2.093541,-1.425491,1.0,1.0
12369,21662,-18.018561,10.558600,-24.667741,11.786180,-10.564657,-2.645681,-18.023468,2.693655,-6.219464,-12.744761,9.318138,-13.377857,1.021924,-12.784346,-0.451146,-7.551365,-11.325997,-4.701253,0.516317,1.007703,-2.319479,0.908839,1.352904,-1.059222,0.185751,0.687037,2.078081,-1.429517,1.0,1.0


In [17]:
new_dataset['Class'].value_counts()

0.0    492
1.0     56
Name: Class, dtype: int64

In [18]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,10605.418699,-0.164707,0.244386,0.904851,0.216578,-0.105169,0.104478,-0.122574,-0.005413,1.015294,-0.319995,0.827127,-1.574664,1.041419,0.941358,-0.163075,-0.055654,0.425012,-0.018063,-0.042796,0.029347,-0.085970,-0.161391,-0.038879,0.049987,0.124447,-0.030313,0.018904,0.016173,62.290711
1.0,12262.107143,-4.727948,4.660436,-9.328536,6.783464,-2.890388,-2.030360,-6.578337,1.028374,-2.981468,-7.502753,6.879584,-10.591544,0.548941,-9.973580,-0.256254,-5.225182,-7.478164,-2.516508,0.272975,0.453283,0.116896,0.048132,-0.225526,-0.452514,-0.124184,0.333325,0.793484,-0.046177,90.815893


In [19]:
# Splitting the data into Features & Targets
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [20]:
print(X)

        Time         V1         V2  ...       V27       V28  Amount
6071    6899  -1.614750   1.513623  ...  0.954628  0.398771   39.02
4077    3734   1.362114  -0.510953  ... -0.031541  0.002629   22.82
7381    9950   1.127197   0.044301  ... -0.027920 -0.006466    6.99
5559    5674  -0.547534   1.164591  ...  0.093608  0.085051   15.17
2353    1899  -0.994431   0.719533  ... -0.008284  0.021924  107.94
...      ...        ...        ...  ...       ...       ...     ...
11880  20451 -15.819179   8.775997  ...  2.136424 -1.411945    1.00
12070  20931 -16.367923   9.223692  ...  2.122796 -1.416741    1.00
12108  21046 -16.917468   9.669900  ...  2.108471 -1.421243    1.00
12261  21419 -17.467710  10.114816  ...  2.093541 -1.425491    1.00
12369  21662 -18.018561  10.558600  ...  2.078081 -1.429517    1.00

[548 rows x 30 columns]


In [21]:
print(Y)

6071     0.0
4077     0.0
7381     0.0
5559     0.0
2353     0.0
        ... 
11880    1.0
12070    1.0
12108    1.0
12261    1.0
12369    1.0
Name: Class, Length: 548, dtype: float64


In [22]:
# Split the data into Training data & Testing Data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=105)

In [23]:
print(X.shape, X_train.shape, X_test.shape)

(548, 30) (438, 30) (110, 30)


**Model Training** => *Logistic Regression*

In [24]:
model = LogisticRegression()

In [25]:
# Training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**Model Evaluation** => *Accuracy Score*

In [26]:
# Accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [27]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9885844748858448


In [28]:
# Accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [29]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9636363636363636
